In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json
import time
from requests.exceptions import ConnectionError

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [3]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_63495/4110031066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [4]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

# get the slug from the urls. the slug is the input to the GraphQL API call 
slugs = list(map(get_slug, df_1['main_url'].tolist()))
# while iterating through all projects, add the story and risk to these lists. At the end len of both list 
# should be the same as the len of the dataframe. if all good then add as columns to the dataframe df. 
creator_list = []

In [5]:
# instantiate a new Session 
s = requests.Session()

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

# PARAMS
counter = 0
# pause for 30 seconds every 5 iterations
for i in range(405, 1000):
    slug = slugs[i]
    if counter == 5:
        print("TAKING A NAP ZZZ...")
        time.sleep(30)
        counter = 0
    counter += 1
    print(f"--------{slug}------")
    # POST request to GraphQL API situated in kickstarter domain 
    # response from POST request should contain the story and risks
    connect_try = 0
    not_found = True
    while connect_try < 30 and not_found:       
        try:
            r = s.post("https://www.kickstarter.com/graph",
                headers = {
                  'content-type': 'application/json',
                  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
                  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
                  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
                },
                json = {
                    "variables":{
                        "slug": slug
                    },
                    "query": "query CreatorSection($slug: String!) {\n  me {\n    id\n    name\n  }\n  project(slug: $slug) {\n    id\n    verifiedIdentity\n    creator {\n      id\n      name\n      imageUrl(width: 100)\n      url\n      lastLogin\n      biography\n      isFacebookConnected\n      allowsFollows\n      backingsCount\n      location {\n        displayableName\n      }\n      launchedProjects {\n        totalCount\n      }\n      websites {\n        url\n        domain\n      }\n    }\n    collaborators {\n      edges {\n        node {\n          name\n          imageUrl(width: 200)\n          url\n        }\n        title\n      }\n    }\n  }\n}\n"
                })
            not_found = False
        except ConnectionError:
            print("Connection Error")
            time.sleep(5)
            connect_try += 1
    try:
        result = r.json()
        print("-----CREATOR-----")
        creator_list.append(result['data']['project'])
    except:
        print(r)
        print("TAKING A BIG NAP ZZZ...")
        time.sleep(60*60)
        #try one more time
        r = s.post("https://www.kickstarter.com/graph",
                headers = {
                  'content-type': 'application/json',
                  'x-csrf-token': 'bL5nMRm5s147N8RxvNcth-E8NAOt3SprGYtvvKsVLnuZ3XwQ71cy1zrDfjIAGTCHV0obMRNqRMbBJT7nF2ZSAA',
                  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
                  'cookie': '_ksr_session=eus10UNK7iB%2FwIMXafXg5yJdYvRD7rHlR%2Ba1KEbUnB5EKTgQxNKEIqrp%2BIyjRC%2FNXw76u4Eki4AsiBom5M1LKNJKWCD65CBzeegWrk2%2FRV9Sht6dlOKplCATNdrmEIIa0Bt%2FKKGnT2m9ID1nTCduj73%2BdeJK663MNcbYWXBHkFyZJgUKh4JcCx3hL0UF2quVeBvfV45M0CNrT1j4DchvTlFBn%2BpE37ZaGldNz7BF0RAcCjIZrt%2FzNkxugCaFfWEPjxh9amOCvytIcdjxOaE68AAmHuU%3D--RPNmoWC%2FGnWTmsfw--9466UG%2Bqs2V%2F8B8zFtXUig%3D%3D'
                },
                json = {
                    "variables":{
                        "slug": slug
                    },
                    "query": "query CreatorSection($slug: String!) {\n  me {\n    id\n    name\n  }\n  project(slug: $slug) {\n    id\n    verifiedIdentity\n    creator {\n      id\n      name\n      imageUrl(width: 100)\n      url\n      lastLogin\n      biography\n      isFacebookConnected\n      allowsFollows\n      backingsCount\n      location {\n        displayableName\n      }\n      launchedProjects {\n        totalCount\n      }\n      websites {\n        url\n        domain\n      }\n    }\n    collaborators {\n      edges {\n        node {\n          name\n          imageUrl(width: 200)\n          url\n        }\n        title\n      }\n    }\n  }\n}\n"
                })
        try:
            result = r.json()
            print("-----CREATOR-----")
            creator_list.append(result['data']['project'])
        except:
            print("NAP DIDN't WORK", r)
            break

--------philsamfos-1/il-film-sul-mio-primo-libro------
-----CREATOR-----
--------welshdragonmedia/a-demon-called-grace------
-----CREATOR-----
--------noxcuses/a-new-family-cafe-for-the-car-and-motorbike-enthus------
-----CREATOR-----
--------okrusko/legends-fantasy-violin-live-cd-release-extended------
-----CREATOR-----
--------27398873/up-down-percussions-debut-album------
-----CREATOR-----
TAKING A NAP ZZZ...
--------ibdfilm/ill-be-damned------
-----CREATOR-----
--------bloodbound/bloodbound-a-youtube-series------
-----CREATOR-----
--------joelcosmejr/community-grounds-coffee-and-meeting-house------
-----CREATOR-----
--------knightsandvampires/a-tale-of-knights-and-vampires------
-----CREATOR-----
--------pallanmovie/pallan-the-movie------
-----CREATOR-----
TAKING A NAP ZZZ...
--------86814100/be-my-muse-new-studio-new-life-and-9-paintings------
-----CREATOR-----
--------1216943848/the-rubber-brush------
-----CREATOR-----
--------876959954/soldier-who-wants-to-become-an-artist------

-----CREATOR-----
--------nhamburger/quevos-the-original-egg-white-chips------
-----CREATOR-----
--------1127315403/full-length-album-darkness-and-light------
-----CREATOR-----
--------1116721168/asarum------
-----CREATOR-----
TAKING A NAP ZZZ...
--------851060982/pdq-bach-the-golden-anniversary------
-----CREATOR-----
--------48955335/taps-audio-jack-flow-control------
-----CREATOR-----
--------marsbeet-speaker/bt-speaker-with-rgb-led-light-and-superior-stereophonic-sound------
-----CREATOR-----
--------organicchemisrty/organic-chemistry-20------
-----CREATOR-----
--------374971122/glass-door-zine------
-----CREATOR-----
TAKING A NAP ZZZ...
--------489644816/dusk-to-dawn-journey-of-a-west-coast-insomniac------
-----CREATOR-----
--------tfsashelp/top-flight-safe-and-secure------
-----CREATOR-----
--------1858074492/beans-1------
-----CREATOR-----
--------productoreslaserie/productores------
-----CREATOR-----
--------brettisaacson/the-mecca-tv-pilot------
-----CREATOR-----
TAKING A NAP 

-----CREATOR-----
--------bbz/make-100-midnight-barista-short-comic------
-----CREATOR-----
--------tucoelbuitre/shorts-make-100------
-----CREATOR-----
--------622603550/book-club-precious-rascals-and-doodle-quest------
-----CREATOR-----
--------1668055563/colorado-acts-black-box-painting------
-----CREATOR-----
TAKING A NAP ZZZ...
--------nanomesher/nanosound-cd-upsampling-cd-player-and-audio-extrac------
-----CREATOR-----
--------phonicbloom/glo-is-a-fun-interactive-and-meditative-sound-expe------
-----CREATOR-----
--------639167978/tunnels-infinite-multiple-for-eurorack-synthesizer------
-----CREATOR-----
--------mutualmoralmaterials/japanese-calligraphy-box-set------
-----CREATOR-----
--------1849010926/the-4-skeletons-of-the-apocalypse-1-6th-action-figures------
-----CREATOR-----
TAKING A NAP ZZZ...
--------1470793130/notebook-for-u-for-ukraine------
-----CREATOR-----
--------859562474/uncivil-servants------
-----CREATOR-----
--------1429966657/the-putumayo-project------
-----CRE

-----CREATOR-----
--------1403226937/widdershins-hangmans-knot------
-----CREATOR-----
--------stevedyslexicmccue/unique-dyslexic-eye-podcasting-broadcasting------
-----CREATOR-----
--------2093139132/the-echo-theatre-2015------
-----CREATOR-----
TAKING A NAP ZZZ...
--------1674897781/jessies-super-normal-regular-average-day-a-feature-film------
-----CREATOR-----
--------kaylon-king/kristin-coopers-perfect-high-school-experience------
-----CREATOR-----
--------covertellusk/papeles-higienicos-y-panuelos-desechables-mimeticos------
-----CREATOR-----
--------justinghill/fetchpro-throw-any-dog-toy-sonicfetch-a-whistling-ball------
-----CREATOR-----
--------el-dorado/el-dorado-volume-1------
-----CREATOR-----
TAKING A NAP ZZZ...
--------wargamingrecon/wargaming-recon-2020-podcast-season------
-----CREATOR-----
--------jerrylawler/wrestlings-jerry-the-king-lawler-2-record-set-remastered------
-----CREATOR-----
--------ayoradio/recording-my-own-album------
-----CREATOR-----
--------heartsoulp

-----CREATOR-----
--------mickhayesmusic/mick-hayes-the-muscle-shoals-sessions-ep------
-----CREATOR-----
--------franicevich/maryvale-prep-review-vol-1------
-----CREATOR-----
--------astralwaters/the-astral-waters-review------
-----CREATOR-----
TAKING A NAP ZZZ...
--------1548966511/all-the-rage------
-----CREATOR-----
--------1685810889/grandmas-hidden-gems------
-----CREATOR-----
--------youseeuc/flow-t-brush-a-stylish-and-functional-toothbrush------
-----CREATOR-----
--------scatterbrainhandmade/amelia-and-friends-the-octopals------
-----CREATOR-----
--------919251735/pilot-co-pilots-podcast------
-----CREATOR-----
TAKING A NAP ZZZ...
--------1403226937/widdershins-curtain-call------
-----CREATOR-----
--------1404894924/producedbyizz-debut-album-and-tour------
-----CREATOR-----
--------cirsova/cirsova-vol-1-final-issues-9-and-10------
-----CREATOR-----
--------2136624445/tinted-window-a-new-journal------
-----CREATOR-----
--------bh6ed/monologue-marathon-at-edinburgh-fringe------


-----CREATOR-----
TAKING A NAP ZZZ...
--------candystar/vitality-magazine------
-----CREATOR-----
--------2010091016/the-boiler-journal-biennial------
-----CREATOR-----
--------ironspike/as-the-crow-flies-volume-one------
-----CREATOR-----
--------946731432/oni-x-fox-volume-1-soldier-daze-printed-edition------
-----CREATOR-----
--------epinephrine/don-pat-and-tom-premiere-at-edinburgh-fringe------
-----CREATOR-----
TAKING A NAP ZZZ...
--------360652081/la-piel-de-metal------
-----CREATOR-----
--------lovesweetnothings/loves-sweet-nothings------
-----CREATOR-----
--------awe-edizioni/awe-presenta-samsara-volume-2------
-----CREATOR-----
--------dumbingofage/dumbing-of-age-sixth-book-collection------
-----CREATOR-----
--------181766060/irepresentnw-ad-campaign-and-album------
-----CREATOR-----
TAKING A NAP ZZZ...
--------otcarta/otcarta-productions------
-----CREATOR-----
--------thekey2success/the-key-2-success------
-----CREATOR-----
--------1481393385/daves-delights------
-----CREATOR

# Saving Data

## Latest Iteration Ran Till: 404, Local copy latest: 0 to 404


In [10]:
#SAVE LIST IN LOCAL FOR USE LATER. NAME FILE AS {from}_{to]_{story/risk}.json
creator_file_name = "/Users/ivankoh/Downloads/bt4222 data/405_999_creator" #CHANGE THIS
creator_file_name += ".json"

with open(creator_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(creator_list, f, indent=2) 
    
# with open(story_file_name, 'r') as f:
#     story_subset = json.load(f)

In [6]:
print("creator:", len(creator_list)) 

creator: 595


In [9]:
print("LATEST CALL DF ROW", "\n")
print(df_1.iloc[405+len(creator_list)-1]['creator'])

print("LATEST CREATOR IN LIST", "\n")
print(creator_list[-1])


LATEST CALL DF ROW 

{"id":991035804,"name":"ADV.SOUND","slug":"adv-","is_registered":null,"is_email_verified":null,"chosen_currency":null,"is_superbacker":null,"avatar":{"thumb":"https://ksr-ugc.imgix.net/assets/010/340/701/9d7c77b158f2f25505d4171a12359455_original.png?ixlib=rb-4.0.2&w=40&h=40&fit=crop&v=1581111391&auto=format&frame=1&q=92&s=4bc9de25c5e06b39fb2e6d63f6b5c050","small":"https://ksr-ugc.imgix.net/assets/010/340/701/9d7c77b158f2f25505d4171a12359455_original.png?ixlib=rb-4.0.2&w=80&h=80&fit=crop&v=1581111391&auto=format&frame=1&q=92&s=3b465c00742133115ed1c8c1f44b9dd9","medium":"https://ksr-ugc.imgix.net/assets/010/340/701/9d7c77b158f2f25505d4171a12359455_original.png?ixlib=rb-4.0.2&w=160&h=160&fit=crop&v=1581111391&auto=format&frame=1&q=92&s=d4fd77490d1d292a87b2ba79d55c60b6"},"urls":{"web":{"user":"https://www.kickstarter.com/profile/adv-"},"api":{"user":"https://api.kickstarter.com/v1/users/991035804?signature=1654831362.d5f56b73dbcf0e244c183b8ec2e132048da05a12"}}}
LATEST 

In [165]:
s = requests.Session()

In [17]:
import requests
import json

url = "https://www.kickstarter.com/graph"

payload = json.dumps([
  {
    "variables": {
      "slug": "thehermetichermit/the-transient-mind-tarot"
    },
    "query": "query CreatorSection($slug: String!) {\n  me {\n    id\n    name\n  }\n  project(slug: $slug) {\n    id\n    verifiedIdentity\n    creator {\n      id\n      name\n      imageUrl(width: 100)\n      url\n      lastLogin\n      biography\n      isFacebookConnected\n      allowsFollows\n      backingsCount\n      location {\n        displayableName\n      }\n      launchedProjects {\n        totalCount\n      }\n      websites {\n        url\n        domain\n      }\n    }\n    collaborators {\n      edges {\n        node {\n          name\n          imageUrl(width: 200)\n          url\n        }\n        title\n      }\n    }\n  }\n}\n"
  }
])
headers = {
  'content-type': 'application/json',
  'x-csrf-token': 'fvjxrhwSYir6BG4U761oTRMXCYu4Ql88J4gTJaaXswCLm-qP6vzjo_vw1FdTY3VNpWEmuQb1MZH_JkJ-GuTPew',
  'user-agent': 'Mozilla/5.0 (Linux; U; Android 2.3.4; en-us; Kindle Fire Build/GINGERBREAD) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1',
  'cookie': '_ksr_session=XeMoOdlfAuvBg3vZkCPtR2KPkjnWtoE%2FGrwY68dNgE13OYvjkIxO0Eiaf4Oay2xzILiIHuLsmpCAH%2B9aBGT01%2B7gNDH6exw0dtPhWF5xVSYtZnJawqJNc8IUuF52GC%2FS7QHp1woosgy6PABnOt0X0ITzAvUEg8wwMmv5e1IBYNCmj8ZiCiUwlU2h08X3WX0lQSBSeatHjTEHV%2FgAjhqitXq1xvrNIY7MvvonoTMQat4mSbJKrOzGi74PUMwlhlgHz1oDCWy%2BqAiY%2BIQp9ft4c%2BK2ta8%3D--Rds4wbR2FagzmGoy--ZV4JbIYN8J%2FdS8uWWa5EQw%3D%3D'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.status_code)
print(response.json())
a = response.json()

200
[{'data': {'me': None, 'project': {'id': 'UHJvamVjdC03NTY5NDA0MDI=', 'verifiedIdentity': 'Alexandra Eastman', 'creator': {'id': 'VXNlci0yMDE4MTg3NDcy', 'name': 'Alex E.', 'imageUrl': 'https://ksr-ugc.imgix.net/assets/036/415/186/53fbf1ce977b0312aead1aee65648ff2_original.jpg?ixlib=rb-4.0.2&w=160&h=160&fit=crop&v=1662144462&auto=format&frame=1&q=92&s=ec5c2c4451e85b255cfc03bca9f88b37', 'url': 'https://www.kickstarter.com/profile/thehermetichermit', 'lastLogin': 1664301753, 'biography': 'American living on Canadian soil. \r\nTarot reader for almost 15 years, graphic designer, lover of life.', 'isFacebookConnected': False, 'allowsFollows': True, 'backingsCount': 0, 'location': {'displayableName': 'Bucharest, Romania'}, 'launchedProjects': {'totalCount': 2}, 'websites': []}, 'collaborators': {'edges': []}}}}]


In [65]:
print(a[0]['data']['project'])

{'id': 'UHJvamVjdC03NTY5NDA0MDI=', 'verifiedIdentity': 'Alexandra Eastman', 'creator': {'id': 'VXNlci0yMDE4MTg3NDcy', 'name': 'Alex E.', 'imageUrl': 'https://ksr-ugc.imgix.net/assets/036/415/186/53fbf1ce977b0312aead1aee65648ff2_original.jpg?ixlib=rb-4.0.2&w=160&h=160&fit=crop&v=1662144462&auto=format&frame=1&q=92&s=ec5c2c4451e85b255cfc03bca9f88b37', 'url': 'https://www.kickstarter.com/profile/thehermetichermit', 'lastLogin': 1664301753, 'biography': 'American living on Canadian soil. \r\nTarot reader for almost 15 years, graphic designer, lover of life.', 'isFacebookConnected': False, 'allowsFollows': True, 'backingsCount': 0, 'location': {'displayableName': 'Bucharest, Romania'}, 'launchedProjects': {'totalCount': 2}, 'websites': []}, 'collaborators': {'edges': []}}
